# 🧭 **Pre-entrega**
## 👩‍🎓 *Alumna:* **Romina Galván**
---


---
# ETAPA 1
---

## 1) Crear un Documento en **Google Colaboratory** y cargar los sets de datos como DataFrames



In [38]:
# Importando paquetes necesarios que luego seran llamados con nombres reducidos
import pandas as pd
import numpy as np
from IPython.display import display, HTML
# Montando la carpeta de Drive en Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# Listando el contenido de cierta ruta y chequeando que estan los dataset necesarios para el trabajo
# Dataset necesarios: 'marketing.csv', 'ventas.csv', 'clientes.csv'
import os
os.listdir("/content/drive/MyDrive/datasets")
# Defino las rutas de cada csv para optimizar su acceso
path_ventas    = "/content/drive/MyDrive/datasets/ventas.csv"
path_clientes  = "/content/drive/MyDrive/datasets/clientes.csv"
path_marketing = "/content/drive/MyDrive/datasets/marketing.csv"
# Cargamos los CSV como DataFrames.
df_ventas         = pd.read_csv(path_ventas)
df_clientes       = pd.read_csv(path_clientes)
df_marketing      = pd.read_csv(path_marketing)
# Chequeamos sus dimensiones para comprobar que se cargaron correctamente.
print("Ventas Data Frame Size: ", df_ventas.shape)
print("Clientes Data Frame Size: ", df_clientes.shape)
print("Marketing Data Frame Size: ", df_marketing.shape)
# Mostramos las primeras filas de cada dataset para corroborar estructura de columnas.
# Aplicamos formato de color
display(df_ventas.head(5).style.background_gradient(cmap="Blues"))
display(df_clientes.head(5).style.background_gradient(cmap="Oranges"))
display(df_marketing.head(5).style.background_gradient(cmap="Purples"))

Mounted at /content/drive
Ventas Data Frame Size:  (3035, 6)
Clientes Data Frame Size:  (567, 5)
Marketing Data Frame Size:  (90, 6)


,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro decorativo,$69.94,5.000000,02/01/2024,Decoración
1,811,Lámpara de mesa,$105.10,5.000000,02/01/2024,Decoración
2,1156,Secadora,$97.96,3.000000,02/01/2024,Electrodomésticos
3,1372,Heladera,$114.35,8.000000,02/01/2024,Electrodomésticos
4,1546,Secadora,$106.21,4.000000,02/01/2024,Electrodomésticos


,id_cliente,nombre,edad,ciudad,ingresos
0,1,Aloysia Screase,44,Mar del Plata,42294.680000
1,2,Kristina Scaplehorn,25,Posadas,24735.040000
2,3,Filip Castagne,50,Resistencia,35744.850000
3,4,Liuka Luard,39,Bahía Blanca,27647.960000
4,5,Dore Cockshtt,28,Rosario,28245.650000


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
0,74,Adorno de pared,TV,4.810000,20/03/2024,03/05/2024
1,12,Tablet,RRSS,3.400000,26/03/2024,13/05/2024
2,32,Lámpara de mesa,Email,5.540000,28/03/2024,20/04/2024
3,21,Smartphone,RRSS,6.370000,29/03/2024,16/05/2024
4,58,Alfombra,Email,4.250000,31/03/2024,05/05/2024


## 4- **Introducción a Pandas:**  realizar un análisis exploratorio inicial de los DataFrames.


In [39]:
def eda(df, nombre):
    # Encabezado
    display(HTML(f"""
    <div style="background-color:#2E86C1; color:white; padding:10px; border-radius:8px; font-size:20px;">
        📊 <b>Análisis exploratorio: {nombre}</b>
    </div>
    """))

    # Información básica
    display(HTML(f"""
    <ul style="font-size:14px; line-height:1.6;">
        <li><b>Shape:</b> {df.shape}</li>
        <li><b>Cantidad de columnas:</b> {len(df.columns)}</li>
        <li><b>Columnas:</b> {list(df.columns)}</li>
    </ul>
    """))

    # Tipos de datos
    display(HTML("<h4 style='color:#8E44AD;'>🧠 Tipos de datos</h4>"))
    display(df.dtypes.to_frame("Tipo de dato").T.style.set_table_styles(
        [{'selector':'th', 'props':[('background-color','#D7BDE2')]}]
    ))

    # Elementos Nulos
    display(HTML("<h4 style='color:#E74C3C;'>🚫 Valores nulos por columna</h4>"))
    display(df.isna().sum().to_frame("Nulos").T.style.background_gradient(cmap="Reds"))

    # Primeras filas
    display(HTML("<h4 style='color:#1ABC9C;'>👀 Primeras 5 filas</h4>"))
    display(df.head(5).style.background_gradient(cmap="Blues").set_caption(f"{nombre} - Primeras filas"))

    # Describe
    display(HTML("<h4 style='color:#F39C12;'>📈 Estadísticas descriptivas (numéricas)</h4>"))
    display(df.describe(include='number').style.background_gradient(cmap="Reds").set_caption(f"{nombre} - Describe"))

    # Línea de cierre
    display(HTML("<hr style='border:1px solid #ccc; margin:20px 0;'>"))



In [40]:
eda(df_ventas, "VENTAS DATA FRAME")
eda(df_clientes, "CLIENTES DATA FRAME")
eda(df_marketing, "MARKETING DATA FRAME")


,id_venta,producto,precio,cantidad,fecha_venta,categoria
Tipo de dato,int64,object,object,float64,object,object


,id_venta,producto,precio,cantidad,fecha_venta,categoria
Nulos,0,0,2,2,0,0


,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro decorativo,$69.94,5.000000,02/01/2024,Decoración
1,811,Lámpara de mesa,$105.10,5.000000,02/01/2024,Decoración
2,1156,Secadora,$97.96,3.000000,02/01/2024,Electrodomésticos
3,1372,Heladera,$114.35,8.000000,02/01/2024,Electrodomésticos
4,1546,Secadora,$106.21,4.000000,02/01/2024,Electrodomésticos


,id_venta,cantidad
count,3035.000000,3033.000000
mean,1499.851400,6.496538
std,866.465379,3.457250
min,1.000000,1.000000
25%,748.500000,3.000000
50%,1502.000000,7.000000
75%,2249.500000,9.000000
max,3000.000000,12.000000


,id_cliente,nombre,edad,ciudad,ingresos
Tipo de dato,int64,object,int64,object,float64


...

,id_cliente,edad,ingresos
count,567.000000,567.000000,567.000000
mean,284.000000,37.940035,34668.739012
std,163.823075,10.202885,12974.531446
min,1.000000,20.000000,170.290000
25%,142.500000,30.000000,26015.240000
50%,284.000000,37.000000,35066.830000
75%,425.500000,43.000000,42457.100000
max,567.000000,81.000000,88053.010000


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
Tipo de dato,int64,object,object,float64,object,object


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
Nulos,0,0,0,0,0,0


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
0,74,Adorno de pared,TV,4.810000,20/03/2024,03/05/2024
1,12,Tablet,RRSS,3.400000,26/03/2024,13/05/2024
2,32,Lámpara de mesa,Email,5.540000,28/03/2024,20/04/2024
3,21,Smartphone,RRSS,6.370000,29/03/2024,16/05/2024
4,58,Alfombra,Email,4.250000,31/03/2024,05/05/2024


,id_campanha,costo
count,90.000000,90.000000
mean,45.500000,4.928667
std,26.124701,0.947750
min,1.000000,2.950000
25%,23.250000,4.372500
50%,45.500000,4.900000
75%,67.750000,5.562500
max,90.000000,7.390000


---
## 5- **Calidad de Datos:** Identificar valores nulos y duplicados en los conjuntos de datos. Documentar el estado inicial de los datos.

In [41]:
# Sugerencia: usar .isna().sum() y .duplicated().sum().
# Documentar las observaciones con print() o comentarios.

def calidad(df, nombre, clave=None):
    """
    Analiza la calidad de un DataFrame:
      - Muestra la cantidad de valores nulos por columna.
      - Informa cuántas filas están completamente duplicadas.
      - Si se indica una clave, analiza duplicados en esa columna.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame a analizar.
    nombre : str
        Nombre o descripción del DataFrame (ejemplo: 'VENTAS').
    clave : str, opcional
        Nombre de la columna donde se desea verificar duplicados específicos.
    """

    print(f"\n{'='*80}")
    print(f"📊 Análisis de calidad del DataFrame: **{nombre.upper()}**")
    print(f"{'='*80}\n")

    # -------------------------------------------------
    # 1. Conteo de valores nulos
    # -------------------------------------------------
    nulos = df.isna().sum().to_frame("Nulos")
    print("🔍 Valores nulos por columna:")
    display(nulos)

    total_nulos = int(nulos["Nulos"].sum())
    print(f"Total de valores nulos en el DataFrame: {total_nulos:,}\n")

    # -------------------------------------------------
    # 2. Filas duplicadas completas
    # -------------------------------------------------
    dup_rows = df.duplicated(keep=False).sum()
    print(f"📑 Filas completamente duplicadas: {dup_rows}\n")

    # -------------------------------------------------
    # 3. Duplicados por clave (si se indicó)
    # -------------------------------------------------
    if clave:
        if clave in df.columns:
            dup_key = df[clave].duplicated(keep=False).sum()
            print(f"🔁 Duplicados en la columna clave '{clave}': {dup_key}")

            if dup_key > 0:
                duplicados_ordenados = (
                    df[df[clave].duplicated(keep=False)][clave]
                    .value_counts()
                    .sort_values(ascending=False)
                )

                print("\nTop 10 valores duplicados más frecuentes:")
                display(duplicados_ordenados.head(10).to_frame("Frecuencia"))
            else:
                print(f"✅ No se encontraron duplicados en la clave '{clave}'.")
        else:
            print(f"⚠️ La clave '{clave}' no existe en el DataFrame.")
    else:
        print("ℹ️ No se indicó una columna clave para análisis de duplicados.\n")

    # -------------------------------------------------
    # 4. Resumen final
    # -------------------------------------------------
    print("\n" + "=" * 70)
    print(f"{'RESUMEN DE CONTROL DE CALIDAD':^70}")
    print("=" * 70)
    print(f"{'Total de valores nulos':<40} → {total_nulos:,}")
    print(f"{'Filas duplicadas':<40} → {dup_rows}")
    if clave and clave in df.columns:
      print(f"{f'Duplicados en columna \"{clave}\"':<40} → {dup_key}")
    print("=" * 70 + "\n")



In [42]:
calidad(df_ventas, "VENTAS DATA FRAME", clave="id_venta")
calidad(df_clientes, "CLIENTES DATA FRAME", clave="id_cliente")
calidad(df_marketing, "MARKETING DATA FRAME", clave="id_campanha")


📊 Análisis de calidad del DataFrame: **VENTAS DATA FRAME**

🔍 Valores nulos por columna:


,Nulos
id_venta,0
producto,0
precio,2
cantidad,2
fecha_venta,0
categoria,0


Total de valores nulos en el DataFrame: 4

📑 Filas completamente duplicadas: 70

🔁 Duplicados en la columna clave 'id_venta': 70

Top 10 valores duplicados más frecuentes:


,Frecuencia
id_venta,
56,2
421,2
424,2
1868,2
2545,2
2778,2
145,2
300,2
439,2



                    RESUMEN DE CONTROL DE CALIDAD                     
Total de valores nulos                   → 4
Filas duplicadas                         → 70
Duplicados en columna "id_venta"         → 70


📊 Análisis de calidad del DataFrame: **CLIENTES DATA FRAME**

🔍 Valores nulos por columna:


,Nulos
id_cliente,0
nombre,0
edad,0
ciudad,0
ingresos,0


Total de valores nulos en el DataFrame: 0

📑 Filas completamente duplicadas: 0

🔁 Duplicados en la columna clave 'id_cliente': 0
✅ No se encontraron duplicados en la clave 'id_cliente'.

                    RESUMEN DE CONTROL DE CALIDAD                     
Total de valores nulos                   → 0
Filas duplicadas                         → 0
Duplicados en columna "id_cliente"       → 0


📊 Análisis de calidad del DataFrame: **MARKETING DATA FRAME**

🔍 Valores nulos por columna:


,Nulos
id_campanha,0
producto,0
canal,0
costo,0
fecha_inicio,0
fecha_fin,0


Total de valores nulos en el DataFrame: 0

📑 Filas completamente duplicadas: 0

🔁 Duplicados en la columna clave 'id_campanha': 0
✅ No se encontraron duplicados en la clave 'id_campanha'.

                    RESUMEN DE CONTROL DE CALIDAD                     
Total de valores nulos                   → 0
Filas duplicadas                         → 0
Duplicados en columna "id_campanha"      → 0



---
# ETAPA 2
---

## 1- **Limpieza de Datos:** Limpiar el conjunto de datos eliminando duplicados y caracteres no deseados. Documentar el proceso y los resultados.

### Pasos realizados:
- Creamos una copia de los datasets originales.
- Eliminamos duplicados.
- Normalizamos texto en columnas tipo `object` (trim + capitalización simple).
- Convertimos columnas de fecha a tipo fecha (`datetime`).
- Convertimos precio y cantidad a valores numéricos, si existen.
- Guardamos los CSV limpios para análisis posterior.

In [43]:
# Sugerencia: aplicar .drop_duplicates(), .str.strip(), .str.replace() según el caso.
# Documentar el proceso y los resultados.
# ============================================
# 🧹 LIMPIEZA Y NORMALIZACIÓN DE LOS DATASETS
# ============================================

# -------------------------------------------------
# 1. Crear copias independientes para no modificar los originales
# -------------------------------------------------
ventas_clean = df_ventas.copy()
clientes_clean = df_clientes.copy()
marketing_clean = df_marketing.copy()

# -------------------------------------------------
# 2. Eliminar filas completamente duplicadas
# -------------------------------------------------
ventas_clean = ventas_clean.drop_duplicates()
clientes_clean = clientes_clean.drop_duplicates()
marketing_clean = marketing_clean.drop_duplicates()

In [44]:
# Chequeo que en mi nuevo df no hay duplicados
calidad(ventas_clean, "VENTAS DATA FRAME LIMPIO", clave="id_venta")



📊 Análisis de calidad del DataFrame: **VENTAS DATA FRAME LIMPIO**

🔍 Valores nulos por columna:


,Nulos
id_venta,0
producto,0
precio,2
cantidad,2
fecha_venta,0
categoria,0


Total de valores nulos en el DataFrame: 4

📑 Filas completamente duplicadas: 0

🔁 Duplicados en la columna clave 'id_venta': 0
✅ No se encontraron duplicados en la clave 'id_venta'.

                    RESUMEN DE CONTROL DE CALIDAD                     
Total de valores nulos                   → 4
Filas duplicadas                         → 0
Duplicados en columna "id_venta"         → 0



In [45]:
# -------------------------------------------------
# 3. Función para limpiar texto en columnas tipo string
# -------------------------------------------------
# Descripción de pasos:
# 1. Selecciona todas las columnas de tipo objeto (strings).
# 2. Convierte los valores a string.
# 3. Elimina espacios al inicio y al final de cada celda.
# 4. Quita caracteres invisibles como tabulaciones, saltos de línea, etc.
# 5. Reemplaza espacios consecutivos por uno solo.
# 6. Cambia la primera letra de cada palabra (Capitalizar).

def normalizar_texto(df):
    """
    Limpia y normaliza columnas de texto (tipo object) en un DataFrame.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame que contiene columnas tipo string a normalizar.

    Retorna
    -------
    df : pandas.DataFrame
        DataFrame con las columnas de texto normalizadas:
        - Espacios al inicio/final eliminados
        - Caracteres invisibles removidos
        - Múltiples espacios reemplazados por uno
        - Primer letra de cada palabra capitalizada
    """
    for col in df.select_dtypes(include="object").columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r"[\u200b\t\r\n]", "", regex=True)
            .str.replace(" +", " ", regex=True)
            .str.title()
        )

    return df

In [46]:
# -------------------------------------------------
# 4. Función para normalizar columnas de fecha en un DataFrame
# -------------------------------------------------
# Descripción de pasos:
# 1. Recorre todas las columnas del DataFrame.
# 2. Convierte los nombres de las columnas a minuscula.
# 3. Si el nombre de la columna contiene "fecha" o "date":
#    - Convierte los valores a datetime de pandas.
#    - Valores inválidos se convierten en NaT.
#    - Se interpreta día primero (DD/MM/YYYY).

def normalizar_fechas_df(df):
    """
    Convierte automáticamente a datetime las columnas que contienen 'fecha' o 'date'
    (no distingue entre mayúsculas y minúsculas).

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame a normalizar.

    Retorna
    -------
    df : pandas.DataFrame
        DataFrame con columnas de fecha normalizadas.
    """
    for col in df.columns:
        col_lower = col.lower()
        if "fecha" in col_lower or "date" in col_lower:  # Detecta columnas que contengan 'fecha' o 'date'
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)
    return df

In [47]:
# -------------------------------------------------
# 5. Función para limpiar y normalizar columnas numéricas
# -------------------------------------------------
# Descripción de pasos:
# 1. Convierte los valores a string (por si hay mezclas de tipos).
# 2. Elimina símbolos no numéricos como "$".
# 3. Elimina separadores de miles (",").
# 4. Elimina espacios sobrantes.
# 5. Convierte la columna a tipo numérico (float), valores inválidos → NaN.

def normalizar_numeros(df, columnas):
    """
    Limpia y normaliza columnas numéricas representadas como texto en un DataFrame.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame que contiene las columnas a normalizar.
    columnas : list of str
        Lista con los nombres de las columnas a limpiar y convertir a números.

    Retorna
    -------
    df : pandas.DataFrame
        DataFrame con las columnas numéricas normalizadas.
    """
    for col in columnas:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)
                .str.replace("$", "", regex=False)
                .str.replace(",", "", regex=False)
                .str.strip()
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")  # Convertir a float, valores inválidos → NaN
    return df

In [48]:
# Limpieza de los datos!

# 1️⃣ Limpiar texto
# df = normalizar_texto(df)
ventas_clean    = normalizar_texto(ventas_clean)
clientes_clean  = normalizar_texto(clientes_clean)
marketing_clean = normalizar_texto(marketing_clean)

# 2️⃣ Normalizar fechas
# df = normalizar_fechas_df(df)
ventas_clean    = normalizar_fechas_df(ventas_clean)
clientes_clean  = normalizar_fechas_df(clientes_clean)
marketing_clean = normalizar_fechas_df(marketing_clean)

# 3️⃣ Normalizar columnas numéricas
# df = normalizar_numeros(df, ["precio", "cantidad"])
ventas_clean    = normalizar_numeros(ventas_clean, ["precio", "cantidad"])
clientes_clean  = normalizar_numeros(clientes_clean, ["ingresos"])
marketing_clean = normalizar_numeros(marketing_clean, ["costo"])

In [53]:
#mostramos los df luego de normalizar los textos para revisar que queden bien
print(ventas_clean.head(5))
print(clientes_clean.head(5))
print(marketing_clean.head(5))

#chequeamos los tipos de formato
print(ventas_clean.dtypes)
print(clientes_clean.dtypes)
print(marketing_clean.dtypes)

   id_venta           producto  precio  cantidad fecha_venta  \
0       792  Cuadro Decorativo   69.94       5.0  2024-01-02   
1       811    Lámpara De Mesa  105.10       5.0  2024-01-02   
2      1156           Secadora   97.96       3.0  2024-01-02   
3      1372           Heladera  114.35       8.0  2024-01-02   
4      1546           Secadora  106.21       4.0  2024-01-02   

           categoria  
0         Decoración  
1         Decoración  
2  Electrodomésticos  
3  Electrodomésticos  
4  Electrodomésticos  
   id_cliente               nombre  edad         ciudad  ingresos
0           1      Aloysia Screase    44  Mar Del Plata  42294.68
1           2  Kristina Scaplehorn    25        Posadas  24735.04
2           3       Filip Castagne    50    Resistencia  35744.85
3           4          Liuka Luard    39   Bahía Blanca  27647.96
4           5        Dore Cockshtt    28        Rosario  28245.65
   id_campanha         producto  canal  costo fecha_inicio  fecha_fin
0          

In [54]:
# -------------------------------------------------
# 6. Cambio de formato
# -------------------------------------------------
# 🧮 Campo "cantidad" cambio el formato, de float a int64
if "cantidad" in ventas_clean.columns:
    ventas_clean["cantidad"] = pd.to_numeric(
        ventas_clean["cantidad"], errors="coerce"
    ).astype("Int64")
    # .astype("Int64") usa el tipo entero de pandas que permite valores nulos (NaN)

In [55]:
print(ventas_clean.dtypes)

id_venta                int64
producto               object
precio                float64
cantidad                Int64
fecha_venta    datetime64[ns]
categoria              object
dtype: object


In [56]:
# -------------------------------------------------
# 7. Guardar los DataFrames limpios como CSV
# -------------------------------------------------
ventas_clean.info()
ventas_clean.to_csv("/content/drive/MyDrive/datasets/ventas_clean.csv", index=False)
clientes_clean.to_csv("/content/drive/MyDrive/datasets/clientes_clean.csv", index=False)
marketing_clean.to_csv("/content/drive/MyDrive/datasets/marketing_clean.csv", index=False)

print("✅ Archivos guardados: ventas_clean.csv, clientes_clean.csv, marketing_clean.csv")

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, 0 to 3034
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_venta     3000 non-null   int64         
 1   producto     3000 non-null   object        
 2   precio       2998 non-null   float64       
 3   cantidad     2998 non-null   Int64         
 4   fecha_venta  3000 non-null   datetime64[ns]
 5   categoria    3000 non-null   object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 167.0+ KB
✅ Archivos guardados: ventas_clean.csv, clientes_clean.csv, marketing_clean.csv


In [58]:
# ============================================
# 📊 REPORTE GLOBAL DE CALIDAD DE DATOS
# ============================================
# Esta función lee los tres DataFrames limpios (o los recibe en memoria)
# y muestra un resumen comparativo de nulos, duplicados y tipos de datos.
# ============================================

def reporte_calidad_global(dfs, nombres):
    """
    Crea un resumen de calidad de varios DataFrames.

    Parámetros:
      dfs: lista de DataFrames (por ejemplo [ventas_clean, clientes_clean, marketing_clean])
      nombres: lista de nombres correspondientes (["VENTAS", "CLIENTES", "MARKETING"])
    """
    resumen = []
    #zip-->es una función incorporada de Python que une elementos de dos (o más) iterables
    # —por ejemplo, listas, tuplas o cualquier objeto iterable— en pares ordenados.
    for df, nombre in zip(dfs, nombres):
        nulos = df.isna().sum().sum()                    # Total de valores nulos, no por columnas sino total, por eso el doble sum
        duplicados = df.duplicated(keep=False).sum()     # Total de filas duplicadas
        columnas = len(df.columns)                       # Cantidad de columnas
        filas = len(df)                                  # Cantidad de registros

        resumen.append({
            "Dataset": nombre,
            "Filas": filas,
            "Columnas": columnas,
            "Nulos totales": nulos,
            "Duplicados": duplicados,
        })

    reporte = pd.DataFrame(resumen)
    #display(reporte)
    return reporte

# ============================================
# ✅ Ejemplo de uso
# ============================================
print(reporte_calidad_global([df_ventas, df_clientes, df_marketing], ["VENTAS Original", "CLIENTES Original", "MARKETING Original"]))
print(reporte_calidad_global([ventas_clean, clientes_clean, marketing_clean],["VENTAS Copia   ", "CLIENTES Copia   ", "MARKETING Copia   "]))



              Dataset  Filas  Columnas  Nulos totales  Duplicados
0     VENTAS Original   3035         6              4          70
1   CLIENTES Original    567         5              0           0
2  MARKETING Original     90         6              0           0
              Dataset  Filas  Columnas  Nulos totales  Duplicados
0     VENTAS Copia      3000         6              4           0
1   CLIENTES Copia       567         5              0           0
2  MARKETING Copia        90         6              0           0


## 2- **Transformación de Datos:** Aplicar filtros y transformaciones para crear una tabla de ventas que muestre solo los productos con alto rendimiento.

In [50]:
# TODO: Aplicar filtros y transformaciones para crear una tabla de ventas
# que muestre solo los productos con alto rendimiento. calcular el percentil 80
# y filtrar los productos que superen ese umbral en ventas.
#quantile(0.8)
# Sugerencia: usar .query() o condiciones con operadores lógicos.

---
## 3- **Agregación:** Resumir las ventas por categoría de producto y analizar los ingresos generados.

In [51]:
# TODO: Resumir las ventas por categoría de producto y analizar los ingresos generados.
# Sugerencia: usar .groupby() y .agg() para generar métricas como suma y promedio.

---
## 4- **Integración de Datos: (OPCIONAL)** Combinar los sets de datos de ventas y marketing para obtener una visión más amplia de las tendencias.